In [1]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 36.4/112.6 GB disk)


In [2]:
from ultralytics import YOLO


In [3]:
!pip install Flask pyngrok


In [4]:
!ngrok authtoken 2ml1tvUfzrhhV7fLlwn1ixZUxsy_5ijjeyeRm2hJcqAL3M4XH


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# YOLOv8 Segmentation Flask App

This Flask application allows users to upload images for YOLOv8 segmentation, run the YOLOv8 model on the uploaded image, and display the output predictions. The app uses `pyngrok` to expose the Flask server publicly.

## Requirements
Ensure the following libraries are installed in your environment:

```bash
pip install Flask pyngrok ultralytics


In [ ]:
from flask import Flask, request, jsonify, send_from_directory
import subprocess
import os
from pyngrok import ngrok

app = Flask(__name__)

# Directory to save uploaded images and output images
UPLOAD_FOLDER = '/content/drive/MyDrive/uploads/'
OUTPUT_FOLDER_BASE = '/content/drive/MyDrive/predict/'  # Base folder for output
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

@app.route('/')
def index():
    return '''
        <h1>Upload Image for YOLOv8 Segmentation</h1>
        <form action="/predict" method="post" enctype="multipart/form-data">
            <label for="folder_name">Enter Folder Name:</label>
            <input type="text" id="folder_name" name="folder_name" required>
            <br><br>
            <input type="file" name="image" accept="image/*" required>
            <br><br>
            <input type="submit" value="Upload">
        </form>
    '''

@app.route('/predict', methods=['POST'])
def predict():
    # Check if the folder name and image file were provided
    folder_name = request.form.get('folder_name')
    if 'image' not in request.files or not folder_name:
        return jsonify({"error": "Folder name or file part is missing"}), 400

    file = request.files['image']

    # Check if the file name is empty
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    # Create a directory for the user's predictions
    output_folder = os.path.join(OUTPUT_FOLDER_BASE, folder_name)


    # Save the uploaded file
    image_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(image_path)

    # Define the YOLO command with the dynamic folder name
    yolo_command = (
        f"yolo task=segment mode=predict model=/content/drive/MyDrive/results/30_epochs_pt22/weights/best.pt "
        f"conf=0.30 source={image_path} save_txt=True boxes=False project={OUTPUT_FOLDER_BASE} name={folder_name}"
    )

    print("Executing YOLO command:", yolo_command)  # Debugging line

    try:
        subprocess.run(yolo_command, shell=True, check=True)

        # Prepare output images for display
        output_images = os.listdir(output_folder)
        print("Output images found:", output_images)  # Debugging line

        # Filter only image files
        output_image_urls = [
            f"/predict/{folder_name}/{img}" for img in output_images
            if img.endswith(('.JPG', '.png', '.jpeg'))
        ]

        # Create HTML response with both input and output images side by side
        response_html = f'''
            <div style="background-color: black; padding: 20px; color: white; text-align: center;">
                <h2>Prediction complete!</h2>
                <div style="display: flex; justify-content: center; align-items: center;">
                    <div style="margin-right: 20px;">
                        <h3>Input Image</h3>
                        <img src="/uploads/{file.filename}" width="400" style="border: 2px solid white;">
                    </div>
                    <div>
                        <h3>Predicted Output</h3><br>
        '''
        if output_image_urls:
            for img_url in output_image_urls:
                response_html += f'<img src="{img_url}" width="400" style="border: 2px solid white;">'
        else:
            response_html += '<p>No output images found.</p>'
        response_html += '</div></div></div>'

        return response_html

    except subprocess.CalledProcessError as e:
        return jsonify({"error": str(e)}), 500

@app.route('/predict/<path:folder_name>/<path:filename>')
def serve_output_image(folder_name, filename):
    folder_path = os.path.join(OUTPUT_FOLDER_BASE, folder_name)
    return send_from_directory(folder_path, filename)

# Serve input images
@app.route('/uploads/<path:filename>')
def serve_input_image(filename):
    return send_from_directory(UPLOAD_FOLDER, filename)

# Expose the Flask app on port 5000
public_url = ngrok.connect(5000)
print(f"Your app is running at: {public_url}")

# Start the Flask app
app.run(port=5000, debug=False)


Your app is running at: NgrokTunnel: "https://80b0-35-233-181-86.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:07:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:07:24] "GET /favicon.ico HTTP/1.1" 404 -


Executing YOLO command: yolo task=segment mode=predict model=/content/drive/MyDrive/results/30_epochs_pt22/weights/best.pt conf=0.30 source=/content/drive/MyDrive/uploads/anushka.JPG save_txt=True boxes=False project=/content/drive/MyDrive/predict/ name=meow611


INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:07:46] "POST /predict HTTP/1.1" 200 -


Output images found: ['labels', 'anushka.JPG']


INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:07:47] "GET /uploads/anushka.JPG HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Oct/2024 19:07:47] "GET /predict/meow611/anushka.JPG HTTP/1.1" 200 -


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
